### Analysis for Chen et al., 2024 (PMID: 38981439, GEO: GSE236581)

### Prerequisites

In [ ]:
#Load python
import anndata
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from scipy.stats import median_abs_deviation
import matplotlib.pyplot as plt
import os
import scvi
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)
sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=False)

### Load data

In [ ]:
Chen2024 = sc.read_h5ad('../yourpath')
Chen2024 = Chen2024[Chen2024.obs['anno'].isin(['Myeloid cells'])]
Chen2024 = Chen2024[Chen2024.obs['Response'].isin(['CR', 'PR', 'SD'])]
Chen2024.obs['response'] = Chen2024.obs['Response']
Chen2024.layers["counts"] = Chen2024.X.copy()
sc.pp.normalize_total(Chen2024, target_sum=1e4)
sc.pp.log1p(Chen2024)
Chen2024.raw = Chen2024
sc.pp.neighbors(Chen2024, use_rep='X_scVI')
sc.tl.umap(Chen2024)
sc.tl.leiden(Chen2024, resolution=1.5)

### Annotate myeloid cells

In [ ]:
cell_type = {
    '0': 'Monocytes',
    '1': 'Monocytes',
    '2': 'Monocytes',
    '3': 'Monocytes',
    '4': 'Mast cells',
    '5': 'Monocytes',
    '6': 'Monocytes',
    '7': 'Monocytes',
    '8': 'Macrophages_DC2',
    '9': 'Monocytes',
    '10': 'Monocytes',
    '11': 'Monocytes',
    '12': 'DC2',
    '13': 'Monocytes',
    '14': 'Macrophages_DC2',
    '15': 'Macrophages_DC2',
    '16': 'Monocytes',
    '17': 'pDC',
    '18': 'mregDC',
    '19': 'pDC',
    '20': 'Macrophages_DC2',
    '21': 'DC1',
    '22': 'Neutrophils',
    '23': 'Mast cells',
}

Chen2024.obs['anno_myeloid'] = Chen2024.obs.leiden.map(cell_type)

In [ ]:
Chen2024 = Chen2024[Chen2024.obs['anno_myeloid'].isin(['DC1', 'DC2', 'mregDC', 'Macrophages_DC2','Monocytes'])]
Chen2024_Macrophages_DC2 = Chen2024[Chen2024.obs['anno_myeloid'].isin(['Macrophages_DC2'])]
sc.pp.neighbors(Chen2024_Macrophages_DC2, use_rep='X_scVI')
sc.tl.umap(Chen2024_Macrophages_DC2)
sc.tl.leiden(Chen2024_Macrophages_DC2, resolution=1)

In [ ]:
cell_type = {
    '0': 'DC2',
    '1': 'mo-mac',
    '2': 'mo-mac',
    '3': 'DC2',
    '4': 'mo-mac',
    '5': 'mo-mac',
    '6': 'mo-mac',
    '7': 'mo-mac',
    '8': 'mo-mac',
    '9': 'mo-mac',
    '10': 'mo-mac',
    '11': 'mo-mac',
    '12': 'mo-mac',
    '13': 'mo-mac',
    '14': 'mo-mac',
    '15': 'DC2',
}

Chen2024_Macrophages_DC2.obs['anno_Macrophages_DC2'] = Chen2024_Macrophages_DC2.obs.leiden.map(cell_type)
Chen2024.obs['anno_myeloid'] = Chen2024.obs['anno_myeloid'].astype('object')
Chen2024.obs.loc[Chen2024_Macrophages_DC2.obs.index, 'anno_myeloid'] = Chen2024_Macrophages_DC2.obs['anno_Macrophages_DC2']
Chen2024.obs['anno_myeloid'] = pd.Categorical(Chen2024.obs['anno_myeloid'])

### ChEA-X analysis

In [ ]:
#Tumor
Chen2024_sub = Chen2024[Chen2024.obs['Treatment'] == "II"]
Chen2024_sub = Chen2024_sub[Chen2024_sub.obs['response'].isin(['CR','SD'])]
Chen2024_sub = Chen2024_sub[Chen2024_sub.obs['Tissue'].isin(['Tumor'])]
Chen2024v2_cluster_sub = Chen2024_sub[Chen2024_sub.obs['anno_myeloid'].isin(['mo-mac'])]
sc.tl.rank_genes_groups(Chen2024v2_cluster_sub, 'response', method='wilcoxon')
results = Chen2024v2_cluster_sub.uns['rank_genes_groups']
out = np.array([[0,0,0,0,0]])
for group in results['names'].dtype.names:
    out = np.vstack((out, np.vstack((results['names'][group],
                                     results['scores'][group],
                                     results['pvals_adj'][group],
                                     results['logfoldchanges'][group],
                                     np.array([group]*len(results['names'][group])).astype('object'))).T))
    
markers = pd.DataFrame(out[1:], columns = ['Gene', 'scores', 'pval_adj', 'lfc', 'cluster'])
markers = markers[(markers.pval_adj < 0.05) & (abs(markers.lfc) > 1)]
markers_up_NR = markers[markers['pval_adj'] <= 0.05]
markers_up_NR = markers_up_NR[markers_up_NR['lfc'] >= 0.5]
markers_up_NR = markers_up_NR[markers_up_NR['cluster'] == "SD"]
markers_up_NR = list(markers_up_NR['Gene'])

In [ ]:
import gseapy
names = gseapy.get_library_name()
print(names)

In [ ]:
enr_ChIPX_up = gseapy.enrichr(gene_list=markers_up_NR ,gene_sets=['ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X'],organism='Human', outdir='../yourpath/tumor/',cutoff=0.5)

In [ ]:
#Blood
Chen2024_sub = Chen2024[Chen2024.obs['Treatment'] == "II"]
Chen2024_sub = Chen2024_sub[Chen2024_sub.obs['response'].isin(['CR','SD'])]
Chen2024_sub = Chen2024_sub[Chen2024_sub.obs['Tissue'].isin(['Blood'])]
Chen2024v2_cluster_sub = Chen2024_sub[Chen2024_sub.obs['anno_myeloid'].isin(['Monocytes'])]
sc.tl.rank_genes_groups(Chen2024v2_cluster_sub, 'response', method='wilcoxon')
results = Chen2024v2_cluster_sub.uns['rank_genes_groups']
out = np.array([[0,0,0,0,0]])
for group in results['names'].dtype.names:
    out = np.vstack((out, np.vstack((results['names'][group],
                                     results['scores'][group],
                                     results['pvals_adj'][group],
                                     results['logfoldchanges'][group],
                                     np.array([group]*len(results['names'][group])).astype('object'))).T))
    
markers = pd.DataFrame(out[1:], columns = ['Gene', 'scores', 'pval_adj', 'lfc', 'cluster'])
markers = markers[(markers.pval_adj < 0.05) & (abs(markers.lfc) > 1)]
markers_up_NR = markers[markers['pval_adj'] <= 0.05]
markers_up_NR = markers_up_NR[markers_up_NR['lfc'] >= 0.5]
markers_up_NR = markers_up_NR[markers_up_NR['cluster'] == "SD"]
markers_up_NR = list(markers_up_NR['Gene'])
enr_ChIPX_up = gseapy.enrichr(gene_list=markers_up_NR ,gene_sets=['ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X'],organism='Human', outdir='../yourpath/blood/',cutoff=0.5 )